In [ ]:
import itertools
import logging
logging.getLogger('pgmpy').setLevel(logging.ERROR)
import random
import time
import pandas as pd
from Decom_Tree import *
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pgmpy.utils import get_example_model
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator, LogLocator, LogFormatter
from networkx.algorithms.approximation import treewidth_min_degree,treewidth_min_fill_in

In [ ]:
filelist = [ "child", "alarm", "hailfinder","hepar2","win95pts" ,"pigs"]

clique_sizes_all = {}

for file in filelist:
    print(f"Processing {file}...")
    model = get_example_model(file)

    G = nx.DiGraph()
    G.add_nodes_from(model.nodes)
    G.add_edges_from(model.edges)

    decom = Graph_Decom(G)
    cliques = decom.Decom()  # list of cliques

    clique_sizes_all[file] = [len(c)/len(G.nodes) for c in cliques]

all_clique_sizes = []
for file, sizes in clique_sizes_all.items():
    all_clique_sizes.extend(sizes)

all_clique_sizes = np.array(all_clique_sizes)

In [ ]:
mpl.rcParams.update({
    "font.family": "Times New Roman",
    "mathtext.fontset": "stix",
    "font.size": 11,
    "axes.linewidth": 0.8,
    "xtick.major.width": 0.8,
    "ytick.major.width": 0.8,
    "xtick.direction": "out",
    "ytick.direction": "out",
})

fig, ax = plt.subplots(figsize=(6, 4))

num_bins = 30

# ===============================
# 绘制直方图
# ===============================
counts, bins, patches = ax.hist(
    all_clique_sizes,
    bins=num_bins,
    density=True,
    rwidth=0.82,
    edgecolor="black",
    linewidth=0.6,
    color="0.75"
)

# ===============================
# 坐标轴标签
# ===============================
#ax.set_xlabel(r"Normalized clique size $|C|/|V|$")
ax.set_ylabel("Density")

# ===============================
# 横坐标精细控制
# ===============================
ax.xaxis.set_major_locator(MultipleLocator(0.05))

# ===============================
# 纵坐标 log 轴 + 多副刻度
# ===============================
ax.set_yscale("log")
ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=None, numticks=6))
ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=np.arange(1,10), numticks=50))

# 使用 LogFormatter 显示数字
ax.yaxis.set_major_formatter(LogFormatter(base=10.0, labelOnlyBase=False))
ax.yaxis.set_minor_formatter(LogFormatter(base=10.0, labelOnlyBase=False))

ax.tick_params(axis='y', which='major', length=2)  
ax.tick_params(axis='y', which='minor', length=4)  
ax.tick_params(axis='x', which='major', length=4)
ax.tick_params(axis='x', which='minor', length=2)

# ===============================
# 不显示网格线
# ===============================
ax.grid(False)

# 边框线细化
for spine in ax.spines.values():
    spine.set_linewidth(0.8)

plt.tight_layout()
plt.savefig("clique-size.png", dpi=600, bbox_inches="tight")
plt.show()


In [ ]:
results = []

for file in filelist:
    print(f"Processing {file}...")
    model = get_example_model(file)

    G = nx.DiGraph()
    G.add_nodes_from(model.nodes)
    G.add_edges_from(model.edges)

    MG = nx.moral_graph(G)

    tw_md = treewidth_min_degree(MG)
    tw_mf = treewidth_min_fill_in(MG)

    results.append({
        "Network": file,
        "Treewidth (Min Degree)": tw_md[0],
        "Treewidth (Min Fill-In)": tw_mf[0]
    })

# 转为 DataFrame
df = pd.DataFrame(results)

df